In [34]:
# Dependencies
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
import requests
import json

# Migration inflow by authority

Information about the data can be fount here: https://www.nomisweb.co.uk/sources/census_2021_mig

In [35]:
# Create df for migration into England and Wales
mig_ltla_in_df = pd.read_csv("./Resources/MIG004EW_LTLA_IN.csv")
mig_ltla_in_df.head()


,Lower tier local authorities code,Lower tier local authorities label,Migration LTLA (inflow) (6 categories) code,Migration LTLA (inflow) (6 categories) label,Ethnic group (6 categories) code,Ethnic group (6 categories) label,National Statistics Socio-economic Classification (NS-SeC) (10 categories) code,National Statistics Socio-economic Classification (NS-SeC) (10 categories) label,Count
0,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,-8,Does not apply,0
1,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,1,"L1, L2 and L3: Higher managerial, administrati...",0
2,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,2,"L4, L5 and L6: Lower managerial, administrativ...",0
3,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,3,L7: Intermediate occupations,0
4,E06000001,Hartlepool,-8,Does not apply,-8,Does not apply,4,L8 and L9: Small employers and own account wor...,0


In [36]:
# Countlocal authority labels
in_authority = mig_ltla_in_df.groupby(['Lower tier local authorities code', 'Lower tier local authorities label', 'Migration LTLA (inflow) (6 categories) label'])['Count'].sum().reset_index()
in_authority = in_authority.sort_values(by='Count', ascending=False)

in_authority

,Lower tier local authorities code,Lower tier local authorities label,Migration LTLA (inflow) (6 categories) label,Count
1583,E08000025,Birmingham,Lived at the same address one year ago,1007979
1643,E08000035,Leeds,Lived at the same address one year ago,689265
293,E06000052,Cornwall,Lived at the same address one year ago,504531
1625,E08000032,Bradford,Lived at the same address one year ago,495205
341,E06000060,Buckinghamshire,Lived at the same address one year ago,491753
...,...,...,...,...
1728,E09000013,Hammersmith and Fulham,Does not apply,0
612,E07000077,Uttlesford,Does not apply,0
1086,E07000173,Gedling,Does not apply,0
618,E07000078,Cheltenham,Does not apply,0


In [37]:

migration_categories = in_authority['Migration LTLA (inflow) (6 categories) label'].unique()

# Create a separate bar graph for each category
for category in migration_categories:
    filtered_data = in_authority[in_authority['Migration LTLA (inflow) (6 categories) label'] == category]
    
    fig = px.bar(filtered_data, x='Lower tier local authorities label', y='Count',
                 title=f'Migration LTLA: {category}')
    
    fig.update_layout(xaxis_title='Local Authority', yaxis_title='Count')
    fig.update_xaxes(categoryorder='total ascending')  # Sort authorities by total count
    fig.show()

# API For Authority Areas

Can be found here: https://findthatpostcode.uk/#api

In [40]:
# Define a function to fetch GeoJSON data
def get_geojson(lower_tier_local_authority_code):
    base_url = "https://findthatpostcode.uk/areas/"
    geojson_url = f"{base_url}{lower_tier_local_authority_code}.geojson"
    try:
        response = requests.get(geojson_url)
        if response.status_code == 200:
            global geojson_counter  # Define a global variable
            if 'geojson_counter' not in globals():
                geojson_counter = 1
            print(f"Fetched GeoJSON {geojson_counter} for {lower_tier_local_authority_code}")
            geojson_counter += 1
            return response.json() 
    except Exception as e:
        print(f"Error fetching GeoJSON for {lower_tier_local_authority_code}: {str(e)}")
    return None

# Apply the function to create a new column with GeoJSON data
in_authority['GeoJSON'] = in_authority['Lower tier local authorities code'].apply(get_geojson)

# Convert the 'GeoJSON' values to valid JSON strings
in_authority['GeoJSON'] = in_authority['GeoJSON'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)

# Filter out rows with 'GeoJSON' values as None
in_authority = in_authority[in_authority['GeoJSON'].notna()]

# Convert the 'GeoJSON' strings to Python dictionaries
in_authority['GeoJSON'] = in_authority['GeoJSON'].apply(json.loads)

Fetched GeoJSON 1 for E08000025
Fetched GeoJSON 2 for E08000035
Fetched GeoJSON 3 for E06000052
Fetched GeoJSON 4 for E08000032
Fetched GeoJSON 5 for E06000060
Fetched GeoJSON 6 for E08000019
Fetched GeoJSON 7 for E06000047
Fetched GeoJSON 8 for E06000054
Fetched GeoJSON 9 for E08000003
Fetched GeoJSON 10 for E08000012
Fetched GeoJSON 11 for E08000034
Fetched GeoJSON 12 for E06000023
Fetched GeoJSON 13 for E06000062
Fetched GeoJSON 14 for E06000049
Fetched GeoJSON 15 for E09000008
Fetched GeoJSON 16 for E06000058
Fetched GeoJSON 17 for E09000003
Fetched GeoJSON 18 for E06000059
Fetched GeoJSON 19 for E08000036
Fetched GeoJSON 20 for E06000061
Fetched GeoJSON 21 for E06000050
Fetched GeoJSON 22 for E09000009
Fetched GeoJSON 23 for E06000016
Fetched GeoJSON 24 for E08000028
Fetched GeoJSON 25 for E06000011
Fetched GeoJSON 26 for E09000025
Fetched GeoJSON 27 for E08000010
Fetched GeoJSON 28 for W06000015
Fetched GeoJSON 29 for E09000010
Fetched GeoJSON 30 for E08000027
Fetched GeoJSON 31 

In [42]:
in_authority.to_csv("./Resources/mig_ltla_in_data.csv", index=False)
in_authority

,Lower tier local authorities code,Lower tier local authorities label,Migration LTLA (inflow) (6 categories) label,Count,GeoJSON
0,E08000025,Birmingham,Lived at the same address one year ago,1007979,{'features': [{'geometry': {'coordinates': [[[...
1,E08000035,Leeds,Lived at the same address one year ago,689265,{'features': [{'geometry': {'coordinates': [[[...
2,E06000052,Cornwall,Lived at the same address one year ago,504531,{'features': [{'geometry': {'coordinates': [[[...
3,E08000032,Bradford,Lived at the same address one year ago,495205,{'features': [{'geometry': {'coordinates': [[[...
4,E06000060,Buckinghamshire,Lived at the same address one year ago,491753,{'features': [{'geometry': {'coordinates': [[[...
...,...,...,...,...,...
1981,E09000013,Hammersmith and Fulham,Does not apply,0,{'features': [{'geometry': {'coordinates': [[[...
1982,E07000077,Uttlesford,Does not apply,0,{'features': [{'geometry': {'coordinates': [[[...
1983,E07000173,Gedling,Does not apply,0,{'features': [{'geometry': {'coordinates': [[[...
1984,E07000078,Cheltenham,Does not apply,0,{'features': [{'geometry': {'coordinates': [[[...


In [45]:
# Read the DataFrame from CSV file
in_authority = pd.read_csv("./Resources/mig_ltla_in_data.csv")

# Define a function to correct the GeoJSON strings
def fix_geojson(geojson_str):
    # Replace single quotes with double quotes
    fixed_geojson_str = geojson_str.replace("'", "\"")
    return fixed_geojson_str

# Apply the fix_geojson function to the 'GeoJSON' column
in_authority['GeoJSON'] = in_authority['GeoJSON'].apply(fix_geojson)

# Convert the 'GeoJSON' strings to Python dictionaries
in_authority['GeoJSON'] = in_authority['GeoJSON'].apply(json.loads)

# Create a choropleth map
fig = px.choropleth(in_authority, 
                   geojson=in_authority['GeoJSON'],  # GeoJSON data
                   locations=in_authority['Lower tier local authorities code'],  # Location identifiers
                   featureidkey="features[0].properties.code",  # Key for the lower-tier local authorities code
                   locationmode="geojson-id",
                   color="Count",  # Data for coloring
                   hover_name="Lower tier local authorities label",  # Hover text
                   title="Migration LTLA Choropleth Map",
                   color_continuous_scale="Viridis")

fig.update_geos(fitbounds="locations", visible=False)
fig.show()


JSONDecodeError: Expecting value: line 1 column 21820 (char 21819)